In [16]:
import ross as rs
import numpy as np
import plotly.graph_objects as go
import complete_ross as fct

Q_ = rs.Q_
rs.__version__

'1.5.1'

Just claculate the first rotor, need to have this too take the matrice, that we going to use

In [17]:
material_exo = rs.Material(name="mat_exo", rho=7850, E=2.05 * 10**11, Poisson= 0.3)


Low_pressure_elements = []
low_pressure_seal_elements = []
low_pressure_elements = []

for i in range(8) : 
    Low_pressure_elements.append(rs.ShaftElement(L = 0.1, material=material_exo, n=i, idl=0, odl=0.05))

low_pressure_elements.append(
    rs.DiskElement.from_geometry(n=1, material=material_exo, width=0.02, i_d=0.05, o_d=0.257)
)

low_pressure_elements.append(
    rs.DiskElement.from_geometry(n=7, material=material_exo, width=0.02, i_d=0.05, o_d=0.3)
)

low_pressure_seal_elements.append(rs.BearingElement(n=0, kxx=3e7, kyy=3e7, cxx=0, cyy=0))
low_pressure_seal_elements.append(rs.BearingElement(n=8, kxx=3e7, kyy=3e7, cxx=0, cyy=0))
low_pressure_seal_elements.append(rs.BearingElement(n=6, kxx=2.5*10**7, kyy=2.5*10**7, cxx=0))

rotor_lp = rs.Rotor(
    shaft_elements=Low_pressure_elements,
    bearing_elements=low_pressure_seal_elements,
    disk_elements=low_pressure_elements,
)
rotor_lp.plot_rotor()

In [18]:
hight_pressure_elements = []
hight_pressure_seal_elements = []
hight_pressure_disk_elements = []

for i in range(4) : 
    hight_pressure_elements.append(rs.ShaftElement(L = 0.1, material=material_exo, n=i, idl=0.07, odl=0.08))

hight_pressure_disk_elements.append(
    rs.DiskElement.from_geometry(n=2, material=material_exo, width=0.015, i_d=0.08, o_d=0.46)
)

hight_pressure_seal_elements.append(rs.BearingElement(n=0, kxx=2.5e7, kyy=2.5e7, cxx=0, cyy=0))
hight_pressure_seal_elements.append(rs.BearingElement(n=4, kxx=2.5e7, kyy=2.5e7, cxx=0))
rotor_hp = rs.Rotor(
    shaft_elements=hight_pressure_elements,
    bearing_elements=hight_pressure_seal_elements,
    disk_elements=hight_pressure_disk_elements,
)

rotor_hp.plot_rotor()


Need to create one birotor with two shft two extract the therm K and M, which didn't depend on the gyroscopique effect. And so on the RPM

In [19]:

axial_shaft = [rs.ShaftElement(0.1, 0, 0.05, material=material_exo) for _ in range(8)]


coaxial_shaft = [rs.ShaftElement(0.1, 0.07, 0.08, material=material_exo) for _ in range(4)]



disks = [
    rs.DiskElement.from_geometry(n=1, material=material_exo, width=0.02, i_d=0.05, o_d=0.257),
    rs.DiskElement.from_geometry(n=7, material=material_exo, width=0.02, i_d=0.05, o_d=0.3),
    rs.DiskElement.from_geometry(n=11, material=material_exo, width=0.015, i_d=0.08, o_d=0.46)
]

bearings = [
    rs.BearingElement(0, kxx=3e7, kyy=3e7, cxx=0),
    rs.BearingElement(8, kxx=3e7, kyy=3e7, cxx=0),
    rs.BearingElement(9, kxx=2.5e7, kyy=2.5e7, cxx=0),
    rs.BearingElement(6, n_link=13, kxx=2.5e7, kyy=2.5e7, cxx=0)
]
""
# Création du rotor coaxial
shaft = [axial_shaft, coaxial_shaft]
rotor_coaxial = rs.CoAxialRotor(shaft, disks, bearings)
rotor_coaxial.plot_rotor()

In [20]:
modal = rotor_coaxial.run_modal(speed=Q_(0, "RPM"),num_modes=14)
for i in np.arange(0, 7, 1):
    modal.plot_mode_3d(mode=int(i)).show()

In [21]:
fig = modal.plot_mode_2d(0, frequency_units="rpm", width=600, height=600)
fig.layout.title.text = "First CRITICAL SPEED" + fig.layout.title.text
fig.show()

Extract the diverse matrix

In [22]:
G_lp = rotor_lp.G()
print("Gyrioscopique Low pressure \t",G_lp.shape)
G_hp = rotor_hp.G() * 1.5
print("Gyrioscopique High pressure \t",G_hp.shape)

# campbell = rotor_coaxial.run_campbell(speed_range=Q_(list(range(0, 4500, 50)), "RPM"))


Gyrioscopique Low pressure 	 (36, 36)
Gyrioscopique High pressure 	 (20, 20)


In [23]:
G1 = np.hstack((G_lp, np.zeros((G_lp.shape[0], rotor_coaxial.ndof - G_lp.shape[1]))))
G2 = np.hstack((np.zeros((G_hp.shape[0],rotor_coaxial.ndof - G_hp.shape[1])), G_hp))

G = np.vstack((G1, G2))

In [ ]:

max_spin = 20000/60 * 2 * np.pi
samples = 40
speed_range = np.linspace(0, max_spin, samples)
fct.run_campbell(rotor_coaxial, speed_range, frequencies = 10, Gyro=G, slope_critic_speed=1.5) 

In [29]:
critical_speeds = fct.run_critical_speed(rotor_coaxial, num_modes=  14, Gyro=G, slope=1)
print("w1 = ", critical_speeds["wn"])

w1 =  [ 489.61838543  650.48151527 1164.0800074  1261.06522761 1316.49076465
 1780.92776273 1865.37284085]


In [31]:
critical_speeds = fct.run_critical_speed(rotor_coaxial, num_modes=  20, Gyro=G, slope=1.5)
print("w1 = ", critical_speeds["wn"])

w1 =  [ 341.75507041  414.41490517  838.24910613  863.63156107  871.57926544
 1176.51563064 1261.79053414 1443.93244425 1896.61819437 2200.46054897]


In [ ]:
fct.run_campbell(rotor_lp, speed_range, frequencies = 7, Gyro=G_lp)

In [ ]:
fct.run_campbell(rotor_hp, speed_range, frequencies = 7, Gyro= G_hp, slope_critic_speed=1.5)